# Projet M2 MIAS : Risque de réidentification (Analyse Avancée)

**Auteur :** [Votre Nom]
**Superviseur :** Pr Emmanuel Chazard

## 1. Objectif du projet
Ce notebook propose une évaluation approfondie des risques de réidentification en confrontant plusieurs méthodologies de mesure.

Nous benchmarkons deux approches pour les critères clés :
1.  **Corrélation** : Comparaison entre **Spearman** (rang) et **Cramer's V** (nominal).
2.  **Inférence** : Comparaison entre le **Succès Réel** (vérifié par vérité terrain) et le **Risque Théorique** (basé sur l'homogénéité $l$-diversity).

---

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
from scipy.stats import chi2_contingency

# Configuration graphique
plt.style.use('seaborn-v0_8')

## 2. Chargement des Données
Chargement de la base de référence et des fichiers variants depuis `projets_donnees`.

In [ ]:
DATA_DIR = "projets_donnees"

def load_data(directory):
    data = {}
    ref_path = os.path.join(directory, "connaissances_externes.txt")
    if os.path.exists(ref_path):
        data['reference'] = pd.read_csv(ref_path, sep=None, engine='python')
        print(f"Base référence chargée : {len(data['reference'])} lignes")
    
    files = glob.glob(os.path.join(directory, "out_*.txt"))
    for filepath in files:
        name_key = os.path.basename(filepath).replace('.txt', '')
        try:
            data[name_key] = pd.read_csv(filepath, sep=None, engine='python')
        except Exception as e:
            print(f"Erreur {name_key} : {e}")
    return data

datasets = load_data(DATA_DIR)
df_reference = datasets['reference']

# Vérité Terrain pour l'inférence réelle
if 'out_direct_0' in datasets:
    df_truth = datasets['out_direct_0'].set_index('id_sejour')
elif 'out_sample_0' in datasets:
    df_truth = datasets['out_sample_0'].set_index('id_sejour')
else:
    df_truth = None

## 3. Méthodologie Avancée et Comparaison

### 3.1 Benchmark : Corrélation
Le mélange des colonnes vise à détruire la structure de la base. Pour mesurer cette destruction, nous comparons deux méthodes :

* **Méthode A : Spearman (Rang)**
    * *Principe* : Transforme les catégories en nombres (0, 1, 2...) et mesure la corrélation d'ordre.
    * *Limite* : Impose un ordre arbitraire aux données nominales (ex: 'Cardio' < 'Neuro'), ce qui peut créer de faux signaux de corrélation ou en masquer.

* **Méthode B : Cramer's V (Nominal) [NOUVEAU]**
    * *Principe* : Basé sur le test du Chi-2. Mesure la force d'association entre deux variables catégorielles sans notion d'ordre.
    * *Avantage* : Beaucoup plus rigoureux pour des données médicales (CIM10, Spécialité, Sexe).

### 3.2 Benchmark : Inférence
L'inférence est la capacité à déduire une information sensible.

* **Méthode A : Succès Réel (Vérité Terrain)**
    * *Principe* : On vérifie si la valeur devinée est *réellement* la bonne en regardant le fichier original secret.
    * *Sens* : Mesure la vulnérabilité réelle, "Dieu voit tout".

* **Méthode B : Risque Théorique (Homogénéité) [NOUVEAU]**
    * *Principe* : On regarde les groupes d'individus partageant les mêmes quasi-identifiants ($k$-anonymity groups). On calcule la diversité des maladies dans ces groupes.
    * *Calcul* : Si dans un groupe de 3 personnes, tout le monde a le diabète, l'attaquant est sûr à 100% (Risque = 1). S'il y a 3 maladies différentes, il a 1 chance sur 3 (Risque = 0.33).
    * *Sens* : Mesure la confiance de l'attaquant, sans avoir besoin de la vérité terrain.

In [ ]:
# --- FONCTIONS METRIQUES ---

def cramers_v(x, y):
    """Calcule le V de Cramer pour deux séries catégorielles."""
    confusion_matrix = pd.crosstab(x, y)
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    with np.errstate(divide='ignore', invalid='ignore'):
        return np.sqrt(phi2 / min(k-1, r-1)) if min(k-1, r-1) > 0 else 0

def calculate_structure_score(df_ref, df_anon, method='spearman'):
    """Compare les matrices de corrélation (Spearman ou Cramer)."""
    common_cols = [c for c in df_ref.columns if c in df_anon.columns and c != 'id_sejour']
    if not common_cols: return 0.0
    
    # 1. Calcul des matrices
    if method == 'spearman':
        # Factorisation simple pour Spearman
        df1 = df_ref[common_cols].apply(lambda x: pd.factorize(x)[0])
        df2 = df_anon[common_cols].apply(lambda x: pd.factorize(x)[0])
        mat1 = df1.corr(method='spearman').fillna(0)
        mat2 = df2.corr(method='spearman').fillna(0)
    
    elif method == 'cramer':
        # Calcul coûteux : on itère sur les paires
        mat1 = pd.DataFrame(index=common_cols, columns=common_cols, dtype=float)
        mat2 = pd.DataFrame(index=common_cols, columns=common_cols, dtype=float)
        
        # Pour optimiser, on prend un sous-ensemble si trop de colonnes, ou on fait le calcul complet
        # Ici calcul complet simplifié
        for c1 in common_cols:
            for c2 in common_cols:
                if c1 == c2: 
                    mat1.loc[c1, c2] = 1.0
                    mat2.loc[c1, c2] = 1.0
                else:
                    # On calcule Cramer uniquement si ce n'est pas déjà fait (symétrie)
                    if pd.isna(mat1.loc[c2, c1]):
                        val1 = cramers_v(df_ref[c1], df_ref[c2])
                        mat1.loc[c1, c2] = mat1.loc[c2, c1] = val1
                        val2 = cramers_v(df_anon[c1], df_anon[c2])
                        mat2.loc[c1, c2] = mat2.loc[c2, c1] = val2
    
    # 2. Comparaison (Différence Moyenne)
    diff = (mat1 - mat2).abs().mean().mean()
    # Score : 100% = structure identique, 0% = structure détruite
    return max(0, (1 - diff * 2)) * 100

def calculate_advanced_metrics(df_connaissance, df_publie, qi, sensitive_col, df_truth):
    # --- 1. Corrélation (Benchmark) ---
    corr_spearman = calculate_structure_score(df_connaissance, df_publie, method='spearman')
    # Note: Cramer peut être lent sur grosses bases, on le fait ici pour la démonstration
    corr_cramer = calculate_structure_score(df_connaissance, df_publie, method='cramer')

    if any(c not in df_publie.columns for c in qi): return 0, 0, 0, corr_spearman, corr_cramer

    # --- 2. Individualisation ---
    # Groupement par QI
    grouped = df_publie.groupby(qi)
    counts = grouped.size()
    unique_idx = counts[counts == 1].index
    
    # Taux d'individualisation (Réid)
    df_pub_unique = df_publie.set_index(qi)
    df_pub_unique = df_pub_unique[df_pub_unique.index.isin(unique_idx)].reset_index()
    
    merged = pd.merge(df_pub_unique, df_connaissance, on=qi, how='inner', suffixes=('_pub', '_know'))
    
    risk_reid = 0
    if len(merged) > 0:
        success = merged[merged['id_sejour_pub'] == merged['id_sejour_know']]
        risk_reid = len(success) / len(df_publie) * 100

    # --- 3. Inférence (Benchmark) ---
    risk_inf_real = 0.0
    risk_inf_theo = 0.0

    if sensitive_col in df_publie.columns:
        # A. Inférence Théorique (Perspective Attaquant)
        # Pour chaque groupe (combinaison QI), quelle est la diversité de la variable sensible ?
        # Risque = 1 / nombre_valeurs_uniques_dans_le_groupe
        # On moyenne ce risque sur toute la population
        def calc_group_risk(group):
            n_unique = group[sensitive_col].nunique()
            return 1 / n_unique
        
        # On applique sur tous les groupes (pas que les uniques)
        group_risks = grouped.apply(calc_group_risk)
        # On ré-étend ce risque à chaque individu
        risk_inf_theo = df_publie.set_index(qi).join(group_risks.rename('risk'))['risk'].mean() * 100

        # B. Inférence Réelle (Vérité Terrain)
        if df_truth is not None and sensitive_col in df_truth.columns and len(merged) > 0:
            col_pub = f"{sensitive_col}_pub" if f"{sensitive_col}_pub" in merged.columns else sensitive_col
            try:
                # Vérité pour les individus "trouvés" (merged)
                true_vals = df_truth.loc[merged['id_sejour_know']][sensitive_col].values
                obs_vals = merged[col_pub].values
                risk_inf_real = np.sum(true_vals == obs_vals) / len(merged) * 100
            except: pass
            
    return risk_reid, risk_inf_real, risk_inf_theo, corr_spearman, corr_cramer

## 4. Exécution du Benchmark

Comparaison des indicateurs sur les scénarios définis.

In [ ]:
scenarios = {
    '1_Faible': ["age10","sexe","entree_mode","sortie_mode","entree_date_y","sortie_date_y"],
    '2_Moyen': ["age5","sexe","entree_mode","sortie_mode","entree_date_ym","sortie_date_ym","specialite","chirurgie"],
    '3_Fort': ["age","sexe","entree_mode","sortie_mode","entree_date_ymd","sortie_date_ymd","specialite","chirurgie","diabete","insuffisance_renale","demence"]
}

TARGET = 'liste_diag'
results = []

for name, df_var in datasets.items():
    if name == 'reference': continue
    
    parts = name.split('_')
    if len(parts) >= 3:
        file_type = 'Direct' if 'direct' in name else 'Sample'
        try: lvl = int(parts[-1])
        except: lvl = 0
    else: continue

    for scen, cols in scenarios.items():
        reid, inf_real, inf_theo, c_spear, c_cram = calculate_advanced_metrics(
            df_reference, df_var, cols, TARGET, df_truth
        )
        results.append({
            'Type': file_type, 'Mélange (%)': lvl, 'Scénario': scen,
            'Individualisation': reid,
            'Inférence (Réelle)': inf_real,
            'Inférence (Théorique)': inf_theo,
            'Corrélation (Spearman)': c_spear,
            'Corrélation (Cramer)': c_cram
        })

df_res = pd.DataFrame(results)
display(df_res.head())

## 5. Visualisation des Benchmarks

Nous affichons deux graphiques comparatifs pour mettre en évidence les différences méthodologiques.

In [ ]:
def plot_benchmarks(df):
    if df.empty: return
    
    # On se concentre sur le type 'Direct' pour le benchmark des méthodes
    df_plot = df[df['Type'] == 'Direct']
    
    fig, axes = plt.subplots(1, 2, figsize=(18, 6))
    
    # Graphique 1 : Benchmark Inférence (Réel vs Théorique)
    # On prend le Scénario Moyen pour l'exemple
    subset = df_plot[df_plot['Scénario'] == '2_Moyen']
    axes[0].plot(subset['Mélange (%)'], subset['Inférence (Réelle)'], 
                 label='Succès Réel (Vérité Terrain)', marker='o', linewidth=2)
    axes[0].plot(subset['Mélange (%)'], subset['Inférence (Théorique)'], 
                 label='Risque Théorique (Homogénéité)', marker='x', linestyle='--', linewidth=2)
    axes[0].set_title("Benchmark Inférence : Réalité vs Estimation (Scénario Moyen)")
    axes[0].set_ylabel("%")
    axes[0].set_xlabel("Niveau de Mélange (%)`")
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)

    # Graphique 2 : Benchmark Corrélation (Spearman vs Cramer)
    # Indépendant du scénario, on prend le premier bloc
    subset_corr = df_plot[df_plot['Scénario'] == '1_Faible']
    axes[1].plot(subset_corr['Mélange (%)'], subset_corr['Corrélation (Spearman)'], 
                 label='Spearman (Rang)', marker='o', color='purple')
    axes[1].plot(subset_corr['Mélange (%)'], subset_corr['Corrélation (Cramer)'], 
                 label='Cramer V (Nominal)', marker='s', color='orange')
    axes[1].set_title("Benchmark Corrélation : Impact du type de mesure")
    axes[1].set_ylabel("Conservation de la Structure (%)")
    axes[1].set_xlabel("Niveau de Mélange (%)`")
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)

    plt.show()

plot_benchmarks(df_res)

### Analyse des Différences Méthodologiques

1.  **Inférence (Réel vs Théorique)** :
    * Le **Risque Théorique** (pointillés) reste souvent élevé même quand on mélange, car il se base sur la diversité apparente. Si le mélange crée des groupes homogènes par hasard, l'attaquant *croit* avoir réussi.
    * Le **Succès Réel** (ligne pleine) chute brutalement. C'est la vraie protection : même si l'attaquant est confiant, le mélange a en réalité faussé la donnée sensible.
    * *Conclusion* : Le mélange protège mieux que ce que l'attaquant peut estimer.

2.  **Corrélation (Spearman vs Cramer)** :
    * **Spearman** peut sous-estimer la perte de structure pour des variables purement nominales (Specialité) car il cherche un ordre qui n'existe pas.
    * **Cramer's V** est plus robuste et montre généralement une chute plus fidèle de la qualité statistique globale de la base quand le mélange augmente.